# Weather Data Collection for Formula 1 Races

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import time
start = time.time()

In [3]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time

In [4]:
races = pd.read_csv('./data/races.csv')

In [5]:
races.head()

season  round    circuit_id      lat      long      country        date  \
0    1950      1   silverstone  52.0786  -1.01694           UK  1950-05-13   
1    1950      2        monaco  43.7347   7.42056       Monaco  1950-05-21   
2    1950      3  indianapolis  39.7950 -86.23470          USA  1950-05-30   
3    1950      4    bremgarten  46.9589   7.40194  Switzerland  1950-06-04   
4    1950      5           spa  50.4372   5.97139      Belgium  1950-06-18   

                                                 url  
0  http://en.wikipedia.org/wiki/1950_British_Gran...  
1  http://en.wikipedia.org/wiki/1950_Monaco_Grand...  
2  http://en.wikipedia.org/wiki/1950_Indianapolis...  
3  http://en.wikipedia.org/wiki/1950_Swiss_Grand_...  
4  http://en.wikipedia.org/wiki/1950_Belgian_Gran...

In [6]:
races.shape

(1035, 8)

In [7]:
weather = races.iloc[:,[0,1,2]]

In [8]:
info = []

for link in races.url:
    try:
        df = pd.read_html(link)[0]
        if 'Weather' in list(df.iloc[:,0]):
            n = list(df.iloc[:,0]).index('Weather')
            info.append(df.iloc[n,1])
        else:
            df = pd.read_html(link)[1]
            if 'Weather' in list(df.iloc[:,0]):
                n = list(df.iloc[:,0]).index('Weather')
                info.append(df.iloc[n,1])
            else:
                df = pd.read_html(link)[2]
                if 'Weather' in list(df.iloc[:,0]):
                    n = list(df.iloc[:,0]).index('Weather')
                    info.append(df.iloc[n,1])
                else:
                    df = pd.read_html(link)[3]
                    if 'Weather' in list(df.iloc[:,0]):
                        n = list(df.iloc[:,0]).index('Weather')
                        info.append(df.iloc[n,1])
                    else:
                        driver = webdriver.Chrome()
                        driver.get(link)

                        # click language button
                        button = driver.find_element_by_link_text('Italiano')
                        button.click()
                        
                        clima = driver.find_element_by_xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr[9]/td').text
                        info.append(clima) 
                                
    except:
        info.append('not found')

In [9]:
len(info)

1035

In [10]:
weather['weather'] = info

In [11]:
weather.head()

season  round    circuit_id    weather
0    1950      1   silverstone  not found
1    1950      2        monaco  not found
2    1950      3  indianapolis  not found
3    1950      4    bremgarten  not found
4    1950      5           spa  not found

In [12]:
weather.tail()

season  round  circuit_id    weather
1030    2020     13       imola  not found
1031    2020     14    istanbul  not found
1032    2020     15     bahrain  not found
1033    2020     16     bahrain  not found
1034    2020     17  yas_marina  not found

In [13]:
## Create a Weather Dictionnary 

In [14]:
weather_dict = {'weather_warm': ['soleggiato', 'clear', 'warm', 'hot', 'sunny', 'fine', 'mild', 'sereno'],
               'weather_cold': ['cold', 'fresh', 'chilly', 'cool'],
               'weather_dry': ['dry', 'asciutto'],
               'weather_wet': ['showers', 'wet', 'rain', 'pioggia', 'damp', 'thunderstorms', 'rainy'],
               'weather_cloudy': ['overcast', 'nuvoloso', 'clouds', 'cloudy', 'grey', 'coperto']}

In [15]:
weather_df = pd.DataFrame(columns = weather_dict.keys())

In [16]:
for col in weather_df:
    weather_df[col] = weather['weather'].map(lambda x: 1 if any(i in weather_dict[col] for i in x.lower().split()) else 0)
    

In [17]:
weather_df.head()

weather_warm  weather_cold  weather_dry  weather_wet  weather_cloudy
0             0             0            0            0               0
1             0             0            0            0               0
2             0             0            0            0               0
3             0             0            0            0               0
4             0             0            0            0               0

In [18]:
weather_info = pd.concat([weather, weather_df], axis = 1)

In [19]:
weather_info.shape

(1035, 9)

In [20]:
weather_info.head()

season  round    circuit_id    weather  weather_warm  weather_cold  \
0    1950      1   silverstone  not found             0             0   
1    1950      2        monaco  not found             0             0   
2    1950      3  indianapolis  not found             0             0   
3    1950      4    bremgarten  not found             0             0   
4    1950      5           spa  not found             0             0   

   weather_dry  weather_wet  weather_cloudy  
0            0            0               0  
1            0            0               0  
2            0            0               0  
3            0            0               0  
4            0            0               0

In [21]:
weather_info.tail()

season  round  circuit_id    weather  weather_warm  weather_cold  \
1030    2020     13       imola  not found             0             0   
1031    2020     14    istanbul  not found             0             0   
1032    2020     15     bahrain  not found             0             0   
1033    2020     16     bahrain  not found             0             0   
1034    2020     17  yas_marina  not found             0             0   

      weather_dry  weather_wet  weather_cloudy  
1030            0            0               0  
1031            0            0               0  
1032            0            0               0  
1033            0            0               0  
1034            0            0               0

In [22]:
weather_info.to_csv('./data/weather.csv', index= False)

In [23]:
end = time.time()

In [24]:
import datetime
str(datetime.timedelta(seconds=(end - start)))

'0:00:01.766500'

In [25]:
print(str(end - start)+" seconds")

1.766500473022461 seconds
